## Requesting token payload from our Dataminr Client ID

In [1]:
import requests

#Requesting dmaToken with our token
grant_type = 'api_key'
client_id = '16e7e3e124a04ce3b0488ccf7d07adb8'
client_secret = 'fb58ea24d0404d48873e5eed9b83b31d'
myUrl = 'https://gateway.dataminr.com/auth/2/token'
payload = {'grant_type': grant_type, 'client_id': client_id, 'client_secret': client_secret}
r = requests.post(myUrl, data=payload)
r.text

'{"dmaToken":"ee73fadfd94c4799ac40634a082df7a4","refreshToken":"8ba141332ef445d7a0f88eb9384fb791","expire":1555464427139}'

## Convert the request string to a Dictionary using the json library

The initial call is returned as a string.  We need to convert it to an object so we can pull out the token component

https://www.w3schools.com/js/js_json_objects.asp



In [2]:
import json
tokenDetails = json.loads(r.text)
tokenDetails

{'dmaToken': 'ee73fadfd94c4799ac40634a082df7a4',
 'refreshToken': '8ba141332ef445d7a0f88eb9384fb791',
 'expire': 1555464427139}

## Return the dmaToken to be used for retrieving data

In [3]:
token = tokenDetails["dmaToken"]

## Next, let's pull in our lists that we created in the Dataminr Dashboard.

##### We will run a GET request to obtain our lists and convert our dictionary object to view the lists in our account

In [4]:
list_url = 'https://gateway.dataminr.com/account/1/get_lists'
payload = {'Authorization': 'dmauth {}'.format(token)}
r_lists = requests.get(list_url, headers=payload)
myLists = json.loads(r_lists.text)
myLists['watchlists']

[{'name': 'Topics',
  'value': [{'id': 'topics-topEvents',
    'name': 'Top Events',
    'description': 'Your Flash, Urgent, and Momentum notifications'},
   {'id': 'topics-alerts',
    'name': 'Alerts',
    'description': 'Your Alert notifications'},
   {'id': 'topics-all',
    'name': 'Top Events + Alerts',
    'description': 'All of your notifications'}]},
 {'name': 'Custom Lists',
  'value': [{'id': 'custom-698815',
    'name': 'Crime',
    'properties': {'watchlistColor': 'darkblue'},
    'description': '',
    'numericId': '698815',
    'type': 'CUSTOM'},
   {'id': 'custom-707590',
    'name': 'Disease',
    'properties': {'watchlistColor': 'purple'},
    'description': '',
    'numericId': '707590',
    'type': 'CUSTOM'},
   {'id': 'custom-707509',
    'name': 'Terrorism',
    'properties': {'watchlistColor': 'red'},
    'description': '',
    'numericId': '707509',
    'type': 'CUSTOM'},
   {'id': 'custom-720349',
    'name': 'trump',
    'properties': {'watchlistColor': 'darkb

## Let's refine this further and get the names and id associated with each list item

In [5]:
lists_dict = dict() # Dictionary to store all list names

Topic_Lists = myLists['watchlists'][0]['value']
Custom_Lists = myLists['watchlists'][1]['value']

for item in Topic_Lists:
    lists_dict[item['name']] = item['id']

for item in Custom_Lists:
    lists_dict[item['name']] = item['id']

lists_dict

{'Top Events': 'topics-topEvents',
 'Alerts': 'topics-alerts',
 'Top Events + Alerts': 'topics-all',
 'Crime': 'custom-698815',
 'Disease': 'custom-707590',
 'Terrorism': 'custom-707509',
 'trump': 'custom-720349'}

## Let's extract the ID for the custom list we created

Specify a List name for the varaible, list_pick

In [6]:
# We will pull out the list id from from our dictionary object based on the list name
list_pick = 'Top Events'

list_id = lists_dict[list_pick]

print("The {} list ID is {}".format(list_pick, list_id))

The Top Events list ID is topics-topEvents


## Since we have identified the list ID, lets retrieve items from it

#### We will run a GET request to obtain all the alerts that have been captured in our list. Again, using the JSON library, we will convert them into a structure that is more manageable.

In [10]:
import datetime
import time

current_time = int(time.mktime(datetime.datetime.now().timetuple()) * 1000)
print(current_time)
fiveminutesbefore=(current_time - 1440 * 60 * 1000)
print(fiveminutesbefore)

list_url = 'https://gateway.dataminr.com/alerts/2/get_alert?alertversion=14'
payload = {'Authorization': 'dmauth {}'.format(token)}
r_lists = requests.get(list_url, params={'lists': list_id, 'pagesize': '100','start_time': fiveminutesbefore}, headers=payload)
EventsList = json.loads(r_lists.text)

1555361499000
1555275099000


#### Let's take a look at the first item in the list

In [11]:

EventsList[0]#[(len(EventsList)-1)]

{'alertId': '502744325-1555361342558-3',
 'watchlistsMatchedByType': [{'id': '731077',
   'type': 'topics',
   'name': 'All Topics',
   'userProperties': {'watchlistColor': 'green'}}],
 'availableRelatedAlerts': 0,
 'eventTime': 1555361458348,
 'eventVolume': 1,
 'eventLocation': {'coordinates': [43.7221064, 20.686480699999947],
  'name': 'Iv Kraljevački Bataljon, Kraljevo, Serbia',
  'places': ['ee4f4f3a2057b39fc24cac9afedd97f5',
   'd87d51152c7ff10bc854572539c92f75',
   'e147274eb94fc069af9eca934c4a79af',
   '65b9491275ec4fed596b6dcc831fb381'],
  'radius': 0.28455053833112504},
 'source': {'channels': ['localnews']},
 'post': {'timestamp': 1555361342558,
  'languages': [],
  'media': [],
  'link': 'http://rs.n1info.com/Vesti/a476575/Pozar-u-Kraljevu.html'},
 'caption': 'Fire burns at residential building on Iv Kraljevacki Bataljon in Kraljevo, Serbia: Local News Outlet via N1 Vjesti via N1 Vjesti.',
 'categories': [{'name': 'Disasters and Weather - Fires and Collapses',
   'topicType

## How many items are in our feed?

In [93]:
print("There are {} items in our list".format(len(EventsList)))

There are 50 items in our list


### Let's try pulling alerts based on a keyword query

In [39]:
#Enter a keyword
keyword_query_phrase = 'Fire AND Mcdonalds'

list_url = 'https://gateway.dataminr.com/alerts/2/get_alert?alertversion=14'
payload = {'Authorization': 'dmauth {}'.format(token)}
r_lists = requests.get(list_url, params={'query': keyword_query_phrase, 'pagesize': '100'}, headers=payload)
if not r_lists.ok:
    print("Nothing returned for the keyword query, {}".format(keyword_query_phrase))
Keyword_EventsList = json.loads(r_lists.text)
print("There are {} items in our list. \n\nHere is the first result:".format(len(Keyword_EventsList)))
Keyword_EventsList[0]

There are 1 items in our list. 

Here is the first result:


{'alertId': '1062097155-1553961240000-3',
 'availableRelatedAlerts': 0,
 'eventTime': 1553961300000,
 'eventVolume': 1,
 'eventLocation': {'coordinates': [44.5939875, -79.4236846],
  'name': "McDonald's, 320 Memorial Ave, Orillia, ON L3V 5X6, Canada",
  'places': ['a7d932a696312804b5d9f3a3fd93ffc5',
   '1437da19de6a3aec2772b38b4bcb96eb',
   '0a269a52d33a19cd680c4d33aef9a4af',
   '2f7245ea29c7d5a90bfd48512f971ef0'],
  'radius': 0.11442414542100532},
 'source': {'displayName': 'Muskoka411',
  'entityName': 'Muskoka411',
  'channels': ['localnews']},
 'post': {'timestamp': 1553961240000,
  'text': 'No injuries after a vehicle fire at the McDonalds parking lot on Hwy 12 in  Orillia  @OrilliaFire #OrilliaOPP https://t.co/kTNJBIP78t',
  'languages': [{'position': 0, 'lang': 'en'}],
  'media': [{'type': 'photo',
    'url': 'http://pbs.twimg.com/media/D26wcZdWsAAKENd.jpg',
    'description': 'https://twitter.com/Muskoka411/status/1112020305470472192/photo/1',
    'display_url': 'pic.twitter.co

### We can modify our search to look at alerts between a specific time frame

In [9]:
import datetime
import time
#Enter a keyword
keyword_query_phrase = 'Trump'

current_time = int(time.mktime(datetime.datetime.now().timetuple()) * 1000)
print(current_time)
fiveminutesbefore=(current_time - 60 * 60 * 1000)
print(fiveminutesbefore)

list_url = 'https://gateway.dataminr.com/alerts/2/get_alert?alertversion=14'
payload = {'Authorization': 'dmauth {}'.format(token)}
r_lists = requests.get(list_url, params={'query': keyword_query_phrase, 'pagesize': '100', 'start_time': fiveminutesbefore}, headers=payload)
if not r_lists.ok:
    print("Nothing returned for the keyword query, {}".format(keyword_query_phrase))
Keyword_EventsList = json.loads(r_lists.text)
print("There are {} items in our list. \n\nHere is the first result:".format(len(Keyword_EventsList)))
Keyword_EventsList[0]

1555361478000
1555357878000
There are 1 items in our list. 

Here is the first result:


{'alertId': '1921628651-1555360380000-3',
 'availableRelatedAlerts': 0,
 'eventTime': 1555361003000,
 'eventVolume': 1,
 'eventLocation': {'coordinates': [38.9071923, -77.03687070000001],
  'name': 'Washington, DC, USA',
  'places': ['f66b10a1b6d5d260b3ddb7e7518aa5ac',
   '9a899085d95f03afd45786859efc619c',
   '16948ff4c22b94b2c25fe83d51606db4',
   '0a269a52d33a19cd680c4d33aef9a4af',
   '2f7245ea29c7d5a90bfd48512f971ef0'],
  'radius': 9.034112016365727},
 'source': {'channels': ['news']},
 'post': {'timestamp': 1555361009853,
  'languages': [],
  'media': [],
  'link': 'https://www.cnn.com/2019/04/15/politics/kfile-stephen-moore-larry-kudlow-trump/index.html'},
 'caption': 'Federal Reserve nominee Stephen Moore apologizes for comparing Donald Trump\'s immigration policies to "the worst parts of World War II" in 2015, saying "it was never true": News Outlet via CNN.',
 'subCaption': {'bullets': {}},
 'companies': [],
 'categories': [],
 'sectors': [],
 'alertType': {'id': 'urgent', 'nam

## Lets pull out the location information and create a dictionary to store data that will then be passed into a GIS object.  

In [12]:
Event_Dict = dict()
Event_list_issues = list()

for item in EventsList:
    try:
        rec_id = item['alertId']
        Event_Dict[rec_id] = dict()
        Event_Dict[rec_id]['coordinates'] = item['eventLocation']['coordinates']
        Event_Dict[rec_id]['place'] = item['eventLocation']['name']
        Event_Dict[rec_id]['expanded_alert_url'] = item['expandAlertURL']
        Event_Dict[rec_id]['Alert_ID'] = rec_id
    except:
        Event_list_issues.append(item)

## Enter Credentials and authenticate login to ArcGIS Online Organization

In [13]:
from arcgis.gis import GIS
import getpass
username = "SDocherty_AFC" #Seth's username
#username = "lwright_AFC" # Lyle's Username
password = getpass.getpass("Enter Password: ")
gis = GIS("https://arcgisforcommer.maps.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

Enter Password: ········
Successfully logged in as: SDocherty_AFC


## Let's view this information in a map!

In [16]:
sms = {"color": [255,0,0,255], "size": 5, "type": "esriSMS", "style": "esriSMSCircle"}
map1 = gis.map()
record_list = []
for key, val in Event_Dict.items():
    lat = val['coordinates'][0]
    long = val['coordinates'][1]
    content = val['expanded_alert_url']
    title = val['Alert_ID']
    
    # Prep data for pandas dataframe
    record_list.append([lat, long, title, content])
    
    #Let's draw the content real quick to see what it looks like
    coords = [lat, long]
    map1.draw(coords, symbol=sms)#, popup={'title':title, 'content':content})
    
map1 # You can click on the points to expose information about them.

MapView(layout=Layout(height='400px', width='100%'))

## Let's try the alternative to adding data into the map by creating a feature collection

### With this approach, we can create a layer to perform spatial analysis on.

In [17]:
import pandas as pd
df = pd.DataFrame.from_records(record_list)
df.columns = ['lat', 'long', 'category', 'caption']
incidents = gis.content.import_data(df)
map2 = gis.map()
map2.add_layer(incidents)
map2

MapView(layout=Layout(height='400px', width='100%'))

## Let's publish this layer as a feature collection item in our GIS

In [18]:
incidents_item_properties = {'title': 'Dataminr List Feed',
                        'description':'Demo - Collection of feeds from the {} list in Dataminr'.format(list_pick),
                        'tags': 'arcgis python api, pandas, Dataminr, demo',
                        'text': json.dumps({"featureCollection": {"layers": [dict(incidents.layer)]}}),
                        'type':'Feature Collection'}
incidents_item = gis.content.add(incidents_item_properties)
incidents_item

<Item title:"Dataminr List Feed" type:Feature Collection owner:SDocherty_AFC>